In [1]:
# Librerias necesarias para modelo de regresión logística
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [2]:
import os
import pandas as pd
import numpy as np

# Accedemos a cada train y a cada test tanto de X como de Y, y los dejamos metidos en variables para el siguiente paso!! 
# Los separamos en base path y el resto de la ruta de acceso para tener el código mas legible. 
 
BASE_PATH = "/Users/julian/Desktop/vs code/Regularized_linear_regression_model_for_machine_learning_Project_15_Julian_Lopez/Regularized_linear_regression_model_for_machine_learning_Project_15_Julian_Lopez/data/processed/excels"
TRAIN_PATHS = [
    "X_train_con_outliers.xlsx",
    "X_train_sin_outliers.xlsx",
    "X_train_con_outliers_norm.xlsx",
    "X_train_sin_outliers_norm.xlsx",
    "X_train_con_outliers_scal.xlsx",
    "X_train_sin_outliers_scal.xlsx"
]

# Concatenamos el base path al resto de la ruta de acceso para tener el código completo dentro de la variable que usaremos despues. 
TRAIN_DATASETS = []
for path in TRAIN_PATHS:
    TRAIN_DATASETS.append(
        # pd.read_excel(BASE_PATH + "/" + path)
        pd.read_excel(f"{BASE_PATH}/{path}") 
        # pd.read_excel(os.path.join(BASE_PATH, path))
    )

# Los separamos en base path y el resto de la ruta de acceso para tener el código mas legible. 
TEST_PATHS = [
    "X_test_con_outliers.xlsx",
    "X_test_sin_outliers.xlsx",
    "X_test_con_outliers_norm.xlsx",
    "X_test_sin_outliers_norm.xlsx",
    "X_test_con_outliers_scal.xlsx",
    "X_test_sin_outliers_scal.xlsx"
]

# Concatenamos el base path al resto de la ruta de acceso para tener el código completo dentro de la variable que usaremos despues. 
TEST_DATASETS = []
for path in TEST_PATHS:
    TEST_DATASETS.append(
        pd.read_excel(f"{BASE_PATH}/{path}")
    )

# Concatenamos el base path al resto de la ruta de acceso para tener el código completo dentro de la variable que usaremos despues. 
y_train = pd.read_excel(f"{BASE_PATH}/y_train.xlsx")
y_test = pd.read_excel(f"{BASE_PATH}/y_test.xlsx")

### Linear regression

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Le pasamos los train y los test al modelo , lo entrenamos y metemos los resultados del test en una lista para poder comparar y escoger el mejor. 

results = []

for index, dataset in enumerate(TRAIN_DATASETS):
    print(index)
    model = LinearRegression()
    model.fit(dataset, y_train)
    y_pred_train = model.predict(dataset)
    y_pred_test = model.predict(TEST_DATASETS[index])

    results.append(
        {
            "train": r2_score(y_train, y_pred_train),
            "test": r2_score(y_test, y_pred_test)
        }
    )

results

0
1
2
3
4
5


[{'train': 0.9999004466933061, 'test': 0.9998933391379832},
 {'train': 0.9965598284875342, 'test': 0.9981773581378354},
 {'train': 0.9999004466933061, 'test': 0.9998933391379833},
 {'train': 0.9965598284875342, 'test': 0.9981773581378354},
 {'train': 0.9999004466933061, 'test': 0.9998933391379833},
 {'train': 0.9965598284875342, 'test': 0.9981773581378355}]

#### Al entrenar y testear el modelo de regresión lineal obserbamos que ya seria correcto, eficiente y no habria overfiting.
(Suponinedo que obtenemos un resultado con overfiting, procedemos a poner en practica los dierentes modelos de regresion lineal regularizada)

### Regularized linear regression

#### LASSO

In [4]:
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score

# Le pasamos los train y los test al modelo , lo entrenamos y metemos los resultados del test en una lista para poder comparar y escoger el mejor. 

results = []

for index, dataset in enumerate(TRAIN_DATASETS):
    print(index)
    model = Lasso()
    model.fit(dataset, y_train)
    y_pred_train = model.predict(dataset)
    y_pred_test = model.predict(TEST_DATASETS[index])

    results.append(
        {
            "train": r2_score(y_train, y_pred_train),
            "test": r2_score(y_test, y_pred_test)
        }
    )

results

0
1
2
3
4
5


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.416e+03, tolerance: 1.022e+01
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.505e+03, tolerance: 1.022e+01
  model = cd_fast.enet_coordinate_descent(


[{'train': 0.9983759145314643, 'test': 0.9983669788173025},
 {'train': 0.9938883142224108, 'test': 0.9956301293396687},
 {'train': 0.9752595518098467, 'test': 0.9753939011371597},
 {'train': 0.9712769939698906, 'test': 0.975733518786201},
 {'train': 0.0, 'test': -0.0005856256239342716},
 {'train': 0.33520927497780606, 'test': 0.3382742058034598}]

In [5]:
# Optimización del modelo. 

best_dataset = 0 # Porque de results, el mayor r2_score en train y test.

# Ponemos los hyperparametros según la documentación.  

hyperparams = {
    "alpha": np.logspace(-6, 6, 3),  # Rango de valores de alpha
    "fit_intercept": [True, False],  # Para ajustar el término de intersección
    "max_iter": [1000, 5000],  # Número máximo de iteraciones
    "tol": [1e-4, 1e-3, 1e-2]  # Tolerancia de convergencia
}


from sklearn.model_selection import GridSearchCV 

model = Lasso(random_state = 77) # Volvemos a seleccionar el modelo sin entrenar. 
grid = GridSearchCV(model, hyperparams, scoring = "r2") # Metemos en el gris los nuevos hyperparametros

In [6]:
# Entrenamos el modelo con los nuevos parametros. 

grid.fit(TRAIN_DATASETS[best_dataset], y_train)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.932e+03, tolerance: 7.951e+00
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.931e+03, tolerance: 8.151e+00
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iter

GridSearchCV(estimator=Lasso(random_state=77),
             param_grid={'alpha': array([1.e-06, 1.e+00, 1.e+06]),
                         'fit_intercept': [True, False],
                         'max_iter': [1000, 5000],
                         'tol': [0.0001, 0.001, 0.01]},
             scoring='r2')

In [7]:
# En esta variable podemos ver la mejor optimización encontrada entre esos parametros. 

grid.best_estimator_

Lasso(alpha=np.float64(1e-06), max_iter=5000, random_state=77)

In [8]:
# Predecimos con el nuevo modelo ya entrenado por los unevos parametros.  

final_model = grid.best_estimator_
y_pred_train = final_model.predict(TRAIN_DATASETS[best_dataset])
y_pred_test = final_model.predict(TEST_DATASETS[best_dataset])

In [9]:
score_train = r2_score(y_train, y_pred_train),
score_test = r2_score(y_test, y_pred_test)

print(f'Score train: {score_train}')
print(f'Score test: {score_test}')


Score train: (0.9998871148597469,)
Score test: 0.9998849536781578


#### RIDGE

In [10]:
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

# Le pasamos los train y los test al modelo , lo entrenamos y metemos los resultados del test en una lista para poder comparar y escoger el mejor. 

results = []

for index, dataset in enumerate(TRAIN_DATASETS):
    print(index)
    model = Ridge()
    model.fit(dataset, y_train)
    y_pred_train = model.predict(dataset)
    y_pred_test = model.predict(TEST_DATASETS[index])

    results.append(
        {
            "train": r2_score(y_train, y_pred_train),
            "test": r2_score(y_test, y_pred_test)
        }
    )

results

0
1
2
3
4
5


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=9.599e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


[{'train': 0.9999002346528161, 'test': 0.9998936238428845},
 {'train': 0.9965582988188406, 'test': 0.9981863616547031},
 {'train': 0.9998974055455289, 'test': 0.9998940692516658},
 {'train': 0.9965191979722938, 'test': 0.9983484239203533},
 {'train': 0.9988333246260489, 'test': 0.9988211368330092},
 {'train': 0.9960001088503242, 'test': 0.9980275218603266}]

In [11]:
# Optimización del modelo. 

best_dataset = 0 # Porque de results, el mayor r2_score en train y test.

# Ponemos los hyperparametros según la documentación.  

hyperparams = {
    "alpha": np.logspace(-6, 6, 3),  # Rango de valores de alpha
    "fit_intercept": [True, False],  # Para ajustar el término de intersección
    "max_iter": [1000, 5000],  # Número máximo de iteraciones
    "tol": [1e-4, 1e-3, 1e-2]  # Tolerancia de convergencia
}


from sklearn.model_selection import GridSearchCV 

model = Ridge(random_state = 77) # Volvemos a seleccionar el modelo sin entrenar. 
grid = GridSearchCV(model, hyperparams, scoring = "r2") # Metemos en el gris los nuevos hyperparametros

In [12]:
grid.fit(TRAIN_DATASETS[best_dataset], y_train)

GridSearchCV(estimator=Ridge(random_state=77),
             param_grid={'alpha': array([1.e-06, 1.e+00, 1.e+06]),
                         'fit_intercept': [True, False],
                         'max_iter': [1000, 5000],
                         'tol': [0.0001, 0.001, 0.01]},
             scoring='r2')

In [13]:
# En esta variable podemos ver la mejor optimización encontrada entre esos parametros. 

grid.best_estimator_

Ridge(alpha=np.float64(1e-06), fit_intercept=False, max_iter=1000,
      random_state=77)

In [14]:
# Predecimos con el nuevo modelo ya entrenado por los unevos parametros.  

final_model = grid.best_estimator_
y_pred_train = final_model.predict(TRAIN_DATASETS[best_dataset])
y_pred_test = final_model.predict(TEST_DATASETS[best_dataset])

In [15]:
score_train = r2_score(y_train, y_pred_train),
score_test = r2_score(y_test, y_pred_test)

print(f'Score train: {score_train}')
print(f'Score test: {score_test}')

Score train: (0.999900440224841,)
Score test: 0.9998934971090829


#### ELASTIC NET

In [16]:
from sklearn.linear_model import ElasticNet
from sklearn.metrics import r2_score

# Le pasamos los train y los test al modelo , lo entrenamos y metemos los resultados del test en una lista para poder comparar y escoger el mejor. 

results = []

for index, dataset in enumerate(TRAIN_DATASETS):
    print(index)
    model = ElasticNet()
    model.fit(dataset, y_train)
    y_pred_train = model.predict(dataset)
    y_pred_test = model.predict(TEST_DATASETS[index])

    results.append(
        {
            "train": r2_score(y_train, y_pred_train),
            "test": r2_score(y_test, y_pred_test)
        }
    )

results

0
1
2
3
4
5


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.521e+03, tolerance: 1.022e+01
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.691e+03, tolerance: 1.022e+01
  model = cd_fast.enet_coordinate_descent(


[{'train': 0.9990543050383297, 'test': 0.9990497725591442},
 {'train': 0.9950020660617265, 'test': 0.9969727566679268},
 {'train': 0.9563184865324357, 'test': 0.9576742502232053},
 {'train': 0.9530031251969806, 'test': 0.9572507130575221},
 {'train': 0.2016277691953321, 'test': 0.20508935376694049},
 {'train': 0.44851528345001557, 'test': 0.4502621111198114}]

In [26]:
# Optimización del modelo. 

best_dataset = 0 # Porque de results, el mayor r2_score en train y test.

# Ponemos los hyperparametros según la documentación.  

hyperparams = {
    "alpha": np.logspace(-6, 6, 3),  # Rango de valores de alpha
    "l1_ratio": [0.1, 1],  # Rango de mezcla entre L1 y L2
    "fit_intercept": [True, False],  # Ajuste del término de intersección
    "max_iter": [1000, 5000],  # Número máximo de iteraciones
    "tol": [1e-6, 1e-5]  # Tolerancia de convergencia
}


from sklearn.model_selection import GridSearchCV 

model = ElasticNet(random_state = 77) # Volvemos a seleccionar el modelo sin entrenar. 
grid = GridSearchCV(model, hyperparams, scoring = "r2") # Metemos en el gris los nuevos hyperparametros

In [27]:
grid.fit(TRAIN_DATASETS[best_dataset], y_train)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.003e+00, tolerance: 7.951e-02
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.372e+00, tolerance: 8.151e-02
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iter

GridSearchCV(estimator=ElasticNet(random_state=77),
             param_grid={'alpha': array([1.e-06, 1.e+00, 1.e+06]),
                         'fit_intercept': [True, False], 'l1_ratio': [0.1, 1],
                         'max_iter': [1000, 5000], 'tol': [1e-06, 1e-05]},
             scoring='r2')

In [28]:
# En esta variable podemos ver la mejor optimización encontrada entre esos parametros. 

grid.best_estimator_

ElasticNet(alpha=np.float64(1e-06), l1_ratio=1, max_iter=5000, random_state=77,
           tol=1e-06)

In [29]:
# Predecimos con el nuevo modelo ya entrenado por los unevos parametros.  

final_model = grid.best_estimator_
y_pred_train = final_model.predict(TRAIN_DATASETS[best_dataset])
y_pred_test = final_model.predict(TEST_DATASETS[best_dataset])

In [30]:
score_train = r2_score(y_train, y_pred_train),
score_test = r2_score(y_test, y_pred_test)

print(f'Score train: {score_train}')
print(f'Score test: {score_test}')

Score train: (0.9998974951290555,)
Score test: 0.9998930398733974
